In [1]:
import nfl_data_py as nfl
import pandas as pd
from NFL_Player import *
from Injury import *
import requests
from bs4 import BeautifulSoup
from helperFunctions import *
from googlesearch import search
import re
from datetime import datetime
import concurrent.futures
from tqdm import tqdm
import time
import random
import pickle
import injury_analysis

user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:54.0) Gecko/20100101 Firefox/54.0',
    'Mozilla/5.0 (iPhone; CPU iPhone OS 10_3_1 like Mac OS X) AppleWebKit/603.1.30 (KHTML, like Gecko) Version/10.0 Mobile/14E304 Safari/602.1'
]

/home/ubuntu/FantasyFootballInjuryIQ/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-10-02 21:21:20.192850: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-02 21:21:20.209914: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-02 21:21:20.230519: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-

In [6]:
nflInjuries20 = nfl.import_injuries([2020])
nflInjuries21 = nfl.import_injuries([2021])
nflInjuries22 = nfl.import_injuries([2022])
nflInjuries23 = nfl.import_injuries([2023])
nflInjuries24 = nfl.import_injuries([2024])
allNFLInjuries = pd.concat([nflInjuries20, nflInjuries21, nflInjuries22, nflInjuries23, nflInjuries24])

max_season = max(allNFLInjuries["season"])
max_week = max(allNFLInjuries.loc[(allNFLInjuries["season"] == max_season)]["week"])

historicalNFLInjuries = allNFLInjuries.loc[(allNFLInjuries["season"] != max_season) | (allNFLInjuries["week"] != max_week)].copy()
historicalNFLInjuries['InjuredInGame?'] = historicalNFLInjuries['practice_status'].apply(lambda x: x.strip() == "")

In [45]:
def players_and_injuries_conversion_nfl(historicalNFLInjuries):
    players = {}
    for index, row in historicalNFLInjuries.iterrows():
        keysList = list(players.keys())
        temp_injury = GameInjuryReport(
            row["season"], 
            row["game_type"], 
            row["team"], 
            row["week"], 
            row["report_primary_injury"], 
            row["report_secondary_injury"], 
            row["report_status"], 
            row["practice_primary_injury"], 
            row["practice_secondary_injury"], 
            row["practice_status"], 
            row["InjuredInGame?"]
        )

        if row["full_name"] not in keysList:
            players[row["full_name"]] = Player(row["first_name"], row["last_name"], row["position"])
        players.get(row["full_name"]).add_injury(temp_injury)
    return players

def get_draft_sharks_url(first_name, last_name):
    # Clean up names for URL generation
    first_name_clean = ''.join(char for char in first_name if char.isalnum()).lower()
    last_name_clean = ''.join(char for char in last_name if char.isalnum()).lower()
    full_name = f"{first_name} {last_name}"
    
    # Prepare search query
    search_query = f"{full_name} Draft Sharks"

    # Maximum retries if rate-limited
    max_retries = 5
    retries = 0

    # Start searching
    while retries < max_retries:
        try:
            # Rotate user-agent for each request
            headers = {'User-Agent': random.choice(user_agents)}
            
            # Adding random sleep between 3 and 6 seconds before requests
            random_sleep = random.uniform(3, 6)
            print(f"Waiting {random_sleep:.2f} seconds before making the search request...")
            time.sleep(random_sleep)

            # Perform the search and look for the Draft Sharks URL
            for url in search(search_query, num_results=10):
                if 'draftsharks.com' in url:
                    if f"{first_name_clean}-{last_name_clean}" in url:
                        # Return the injury history URL if found
                        if 'injury-history' in url:
                            return url

                        # If no injury history found, construct it using player ID
                        match = re.search(r'/(\d+)$', url)
                        if match:
                            player_id = match.group(1)
                            return f"https://www.draftsharks.com/fantasy/injury-history/{first_name_clean}-{last_name_clean}/{player_id}"
            break  # Exit if no URL found in this search cycle

        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 429:  # Handle rate limit errors
                wait_time = 2 ** retries + random.uniform(1, 3)  # Exponential backoff
                print(f"Rate limited. Retrying in {wait_time:.2f} seconds...")
                time.sleep(wait_time)
                retries += 1  # Increment retry count
            else:
                print(f"HTTP Error: {e}")
                return None

        except Exception as e:
            print(f"An error occurred: {e}")
            return None

    return None

def add_draftsharks(player, url):
# Maximum retries if rate-limited
    max_retries = 5
    retries = 0

    # Start searching
    while retries < max_retries:
        try:
            print(f"Adding Draftsharks Injuries For {player.first_name} {player.last_name}")
            print(f"Try {retries}/{max_retries}")
            # Rotate user-agent for each request
            headers = {'User-Agent': random.choice(user_agents)}
            
            # Adding random sleep between 3 and 6 seconds before requests
            random_sleep = random.uniform(3, 6)
            print(f"Waiting {random_sleep:.2f} seconds before making the search request...")
            time.sleep(random_sleep)
            print("Getting Response")
            response = requests.get(url, headers= headers)
            print("Response Recieved")
            soup = BeautifulSoup(response.content, 'html.parser')
            table_rows_container = soup.find("div", class_="injury-history-table-container")
            if table_rows_container == None:
                print(f"No Container Response For {player.first_name} {player.last_name}")
                break
            else:
                table_rows_body = table_rows_container.find("tbody")
                if table_rows_body == None:
                    print(f"No Response For {player.first_name} {player.last_name}")
                    break
                else:
                    table_rows = table_rows_body.findAll("tr")
                    if table_rows == None:
                        print(f"No Response For {player.first_name} {player.last_name}")
                        break
                    else:
                        if len(table_rows) > 0:
                            print(f"Response Found For {player.first_name} {player.last_name}")
                        for row in table_rows:
                            row_data = row.findAll("td")
                            injury_date = datetime.strptime(row_data[0].text, '%b %d, %Y')  # Date
                            injury_league = row_data[1].text  # League
                            injury = row_data[2].text  # Injury
                            injury_details = row_data[3].text  # Description
                            temp_injury = DetailedInjuryReport(injury_date, injury, injury_league, injury_details)
                            player.add_injury(temp_injury)
            break

        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 429:  # Handle rate limit errors
                wait_time = 2 ** retries + random.uniform(1, 3)  # Exponential backoff
                print(f"Rate limited. Retrying in {wait_time:.2f} seconds...")
                time.sleep(wait_time)
                retries += 1  # Increment retry count
            else:
                raise Exception(f"HTTP Error: {e}")

        except Exception as e:
            raise Exception(f"An error occurred: {e}")
def players_and_injuries_conversion_draftsharks(player_name, player):
    url = get_draft_sharks_url(player.first_name, player.last_name)
    full_name = player_name
    if url:
        add_draftsharks(player, url)
        print(f"Draft Sharks URL for {full_name}: {url}")
    else:
        print(f"No Draft Sharks URL found for {full_name}.")
def split_dictionary(original_dict, chunk_size=450):
    # Split the original dictionary into smaller chunks
    keys = list(original_dict.keys())
    chunks = [dict((k, original_dict[k]) for k in keys[i:i + chunk_size]) 
              for i in range(0, len(keys), chunk_size)]
    return chunks

In [ ]:
# x = filterHistoricalNFLInjuries(historicalNFLInjuries, position="WR")
playersWithInjuries = players_and_injuries_conversion_nfl(historicalNFLInjuries)
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = {executor.submit(players_and_injuries_conversion_draftsharks, player_name, player): player_name for player_name, player in playersWithInjuries.items()}
    for future in concurrent.futures.as_completed(futures):
        player_name = futures[future]
        try:
            future.result()  # This will raise an exception if the thread raised one
        except Exception as e:
            print(f"Exception occurred for player {player_name}: {e}")


In [ ]:
again = []

# Wrap the playersWithInjuries dictionary with tqdm for progress tracking
for player in tqdm(playersWithInjuries, desc="Processing Players"):
    if not any(isinstance(item, DetailedInjuryReport) for item in playersWithInjuries[player].injuries):
        url = get_draft_sharks_url(playersWithInjuries[player].first_name, playersWithInjuries[player].last_name)
        if url:
            again.append(player)
            print(f"Draft Sharks URL for {player}: {url}")
        else:
            print(f"NO Draft Sharks URL for {player}")
print(len(again))

In [12]:
pattern = r'Draft Sharks URL for [^:]+:\s*(https:\/\/www\.draftsharks\.com\/fantasy\/injury-history\/[^\/]+\/\d+)'
new_urls = [match for line in open('secondPass.txt') for match in re.findall(pattern, line) if match]


In [ ]:
second_chance = playersWithInjuries.copy()
for i in tqdm(range(len(new_urls))):
    print(second_chance[again[i]].injuries)
    add_draftsharks(second_chance[again[i]], new_urls[i])
    print(second_chance[again[i]].injuries)


In [9]:

with open('playersWithInjuries9-30-24.pickle', 'wb') as handle:
    pickle.dump(playersWithInjuries,handle,protocol=pickle.HIGHEST_PROTOCOL)


In [2]:
with open('second_chance10-1-24.pickle', 'rb') as handle:
    second_chance = pickle.load(handle)

In [4]:
all_injuries = []
for player in tqdm(second_chance, desc="Processing Players"):
        injury_dict = second_chance[player].to_dict()
        for d in injury_dict:
            all_injuries.append(d)

all_injuries_df = pd.DataFrame(all_injuries)
all_injuries_df = all_injuries_df.fillna('')
# first_name	last_name	position    injury  report_secondary_injury	report_status	practice_primary_injury	practice_secondary_injury	practice_status	injured_in_game	details
all_injuries_df['Combined'] = all_injuries_df.apply(lambda row: ' '.join(row.drop('injuries').drop('date').fillna('').astype(str)), axis=1)

Processing Players: 100%|██████████| 2751/2751 [00:00<00:00, 57370.23it/s]


In [5]:
with open('playersWithInjurie10-2-24.pickle', 'wb') as handle:
    pickle.dump(all_injuries_df,handle,protocol=pickle.HIGHEST_PROTOCOL)